In [ ]:
!pip install jamdict jamdict-data

     |████████████████████████████████| 203 kB 5.0 MB/s 
     |████████████████████████████████| 53.9 MB 43 kB/s 
     |████████████████████████████████| 70 kB 7.6 MB/s 
  Created wheel for jamdict: filename=jamdict-0.1a11.post2-py3-none-any.whl size=196864 sha256=9680ee5a1d3a95d00bf5ce65943c5107248ad31c0fdb3d8f3a652986a5c47f2a
  Stored in directory: /root/.cache/pip/wheels/99/66/43/c7329e8862594f525e4386c8aea3d98f67f004e4d9518d7ad7
  Created wheel for chirptext: filename=chirptext-0.1.2-py3-none-any.whl size=54261 sha256=b5dee423378e0f74532f29e0247de07929414da637a56c3459e9500733cc3737
  Stored in directory: /root/.cache/pip/wheels/56/fa/c6/364b06b01cea7fc51afe407e89c2388200c2a64842c67ac28e
  Created wheel for puchikarui: filename=puchikarui-0.1-py3-none-any.whl size=10369 sha256=6891cbaed4a2c569443ede5187993df7e08834aec986210064e45dbc03037c78
  Stored in directory: /root/.cache/pip/wheels/85/a9/29/bc97f3598d5d42fa9d6e47fed8187058949c1e893a31021926
  Created wheel for jamdict-data: fil

In [ ]:
import pandas as pd
import numpy as np
import random
from collections import Counter

In [ ]:
wordlist = pd.read_csv('/content/drive/MyDrive/wordlist.csv')

In [ ]:
def mora(word):
    exceptions = 'ぁぃぅぇぉゃょゅァィゥェォャョュ'
    minus = [word.count(char) for char in exceptions]
    return len(word)-sum(minus)

In [ ]:
wordlist = [ [item[0], item[1], mora(item[1])] for item in np.array(wordlist)]

In [ ]:
from jamdict import Jamdict
jam = Jamdict()

In [ ]:
from tqdm import tqdm

In [ ]:
postype = []
for word in tqdm(wordlist):
    try:
        postype.append(jam.lookup(word[0]).entries[0].senses[0].pos[0])
    except:
        postype.append('')

100%|██████████| 2545/2545 [01:35<00:00, 26.68it/s]


In [ ]:
cleanpos = []
kinds = ['pronoun','noun','adverb','adjectival','adjective','verb','counter','expressions','interjection','numeric','suffix']
counter = []
for item in postype:
    if item == '':
        cleanpos.append('')
    else:
        for pos in kinds:
            if pos in item:
                cleanpos.append(pos)
                break


In [ ]:
allwords = np.hstack((np.array(wordlist),np.expand_dims(cleanpos,1)))

In [ ]:
df = pd.DataFrame(allwords)

In [ ]:
df.columns = ['Kanji','Kana','Moras','Type']

In [ ]:
df.loc[(df['Moras'] == '2') & (df['Type'] == 'noun')]

,Kanji,Kana,Moras,Type
48,上,うえ,2,noun
60,下,した,2,noun
67,下手,へた,2,noun
78,世話,せわ,2,noun
85,中,なか,2,noun
...,...,...,...,...
2516,骨,ほね,2,noun
2530,髪,かみ,2,noun
2534,鳥,とり,2,noun
2540,黒,くろ,2,noun


In [ ]:
wordsdf = np.array(df)

In [ ]:
wordsdf[:,2]

array(['2', '3', '3', ..., '3', '4', '2'], dtype=object)

In [ ]:
wordsdf[(wordsdf[:,2] == '3') & (wordsdf[:,3] == 'noun')] 

array([['一人', 'ひとり', '3', 'noun'],
       ['一家', 'いっか', '3', 'noun'],
       ['一致', 'いっち', '3', 'noun'],
       ...,
       ['魚', 'さかな', '3', 'noun'],
       ['黄色', 'きいろ', '3', 'noun'],
       ['黒字', 'くろじ', '3', 'noun']], dtype=object)

In [ ]:
!pip install fasttext

     |████████████████████████████████| 68 kB 3.2 MB/s 
  Using cached pybind11-2.8.1-py2.py3-none-any.whl (208 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3123066 sha256=863c90e0496edc597e5299f9dde9074c51fd26101647ed42cb964f158ff49c61
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
import fasttext

In [ ]:
%%time
import fasttext.util
ft = fasttext.load_model('/content/drive/MyDrive/cc.ja.300.bin')

CPU times: user 4.67 s, sys: 9.1 s, total: 13.8 s
Wall time: 1min 58s


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def compare(word1,word2):
  return cosine_similarity(np.expand_dims(ft.get_word_vector(word1),0),np.expand_dims(ft.get_word_vector(word2),0))[0][0]

def compare_sentence(sentence1,sentence2):
  return cosine_similarity(np.expand_dims(sentence1,0),np.expand_dims(sentence2,0))[0][0]

In [ ]:
compare('櫨','季')

0.19513138

In [ ]:
!git clone https://github.com/brokyo/saijikijs/

Cloning into 'saijikijs'...
remote: Enumerating objects: 1535, done.
remote: Total 1535 (delta 0), reused 0 (delta 0), pack-reused 1535
Receiving objects: 100% (1535/1535), 861.29 KiB | 10.50 MiB/s, done.
Resolving deltas: 100% (443/443), done.


In [ ]:
kigo = pd.read_json('/content/saijikijs/kigo.json')
kigo['subSeason'] = kigo['subSeason'].replace(r'\n','', regex=True)

In [ ]:
from datetime import date
import random

start_date = date.today().replace(day=1, month=1).toordinal()
end_date = date.today().toordinal()

In [ ]:
from datetime import date, datetime

Y = 2000
seasons = [('early spring', (date(Y,  2,  4),  date(Y,  3, 5))),
           ('mid spring', (date(Y,  3,  6),  date(Y,  4, 4))),
           ('late spring', (date(Y,  4, 5),  date(Y,  5, 5))),
           ('early summer', (date(Y,  5, 6),  date(Y,  6, 5))),
           ('mid summer', (date(Y,  6, 6),  date(Y, 7, 6))),
           ('late summer', (date(Y, 7, 7),  date(Y, 8, 7))),
           ('early autumn', (date(Y,  8,  8),  date(Y,  9, 7))),
           ('mid autumn', (date(Y,  9,  8),  date(Y,  10, 7))),
           ('late autumn', (date(Y,  10, 8),  date(Y,  11, 6))),
           ('early winter', (date(Y,  11, 7),  date(Y,  12, 6))),
           ('mid winter', (date(Y,  7, 12),  date(Y, 1, 4))),
           ('late winter', (date(Y, 1, 5),  date(Y, 2, 3)))]

def get_season(day):
    day = day.replace(year=Y)
    return next(season for season, (start, end) in seasons if start <= day <= end)

current_season = get_season(date.today()) #get_season(date.fromordinal(random.randint(start_date, end_date)))
current_kigo = kigo[(kigo['subSeason']==current_season) | (kigo['subSeason']=='all '+current_season.split()[1])]
current_kigo = current_kigo[['japanese','hiragana']]

In [ ]:
df.loc[(df['Moras'] == '2') & (df['Type'] == 'noun')].sample(1)

,Kanji,Kana,Moras,Type
2296,週,しゅう,2,noun


In [ ]:
while 1<2:   
    kigo_choice = np.random.randint(current_kigo.shape[0]-1)
    kigo_kana = list(current_kigo.iloc[[kigo_choice]]['hiragana'])[0]
    kigo_size = mora(kigo_kana)
    if kigo_size<4:
        kigo_choice = list(current_kigo.iloc[[kigo_choice]]['japanese'])[0]
        break
    else:
        pass
print(current_season)
print(kigo_choice)

early winter
蒲団（布団）


In [ ]:
import regex as re

In [ ]:
len(set(kigo['japanese']))

941

In [ ]:
def create_haiku(kigo_choice,kigo_kana,kigo_size):
    close_words = ft.get_nearest_neighbors(kigo_choice,k=200,on_unicode_error='replace')
    

    pattern_hirakan = re.compile(r'([\p{IsHan}\p{IsHira}]+)', re.UNICODE)
    pattern_hiraonly = re.compile(r'([\p{IsHira}]+)', re.UNICODE)
    def verify_hirakan(word):

        hirakan = re.match(pattern_hirakan,word)
        hiraonly = re.match(pattern_hiraonly,word)

        condition = (hirakan != None and hirakan[0] == word) and not (hiraonly != None and hiraonly[0]==word)

        return condition
    todictionary = [word for word in close_words if verify_hirakan(word[1])]
    cleanpos = []

    counter = []
    kinds = ['pronoun','noun','adverb','adjectival','adjective','verb','counter','expressions','interjection','numeric','suffix']
    for item in postype:
        if item == '':
            cleanpos.append('')
        else:
            for pos in kinds:
                if pos in item:
                    cleanpos.append(pos)
                    break
    final_list = []
    
    for word in todictionary:
        try:
            kana = str(jam.lookup(word[1]).entries[0].kana_forms[0])
            pos = jam.lookup(word[1]).entries[0].senses[0].pos[0]
            cleanpos = [item for item in kinds if item in pos][0]

            final_list.append([word[1],kana,mora(kana),cleanpos,word[0]])
        except:
            pass
    df_embeddings = pd.DataFrame(final_list)

    df_final = pd.DataFrame(final_list,columns=['Kanji','Kana','Moras','Type','Similarity'])

    df_pincho = pd.read_csv('/content/drive/MyDrive/freqlist.csv',header=None)
    df_pincho = df_pincho[[1,2,4]]
    df_pincho_raw = df_pincho
    np_pincho = np.array(df_pincho)
    np_pincho = np.array([item for item in np_pincho.tolist() if verify_hirakan(item[0])]) #removing kana only words (lots of onomatopeia and etc)
    df_pincho = pd.DataFrame(np_pincho)
    pincho_wordtypes = np.unique(np_pincho[:,2])
    pincho_translations = ['pronoun','adverb','erase','erase','erase','erase','erase','erase','verb','noun',
                          'noun','noun','noun','noun','noun','numeric','noun','adjective','adjectival','adjectival','adjectival',
                          'interjection','suffix','suffix','suffix','suffix','suffix','erase','erase','erase']
    pincho_dict = dict(zip(pincho_wordtypes,pincho_translations))
    df_pincho = df_pincho.replace({2:pincho_dict})
    pincho_kinds = ['pronoun']+kinds[2:]
    df_pincho = df_pincho.loc[df_pincho[2].isin(pincho_kinds)]
    df_pincho.columns=['Kanji','Kana','Type']
    df_pincho_raw.columns=['Kanji','Kana','Type']
    df_pincho['Moras'] = [mora(word) for word in df_pincho['Kana']]
    df_pincho['Similarity'] = [compare(kigo_choice,word) for word in df_pincho['Kanji']]
    size_of_candidates = 10
    prob_poetic = 0.99

    candidate_nouns = df_final.loc[df_final['Type']=='noun'].sort_values(by='Similarity',ascending=False).head(size_of_candidates)
    candidate_verbs = df_pincho.loc[df_pincho['Type']=='verb'].sort_values(by='Similarity',ascending=False).head(size_of_candidates)
    candidate_adverbs = df_pincho.loc[(df_pincho['Kanji'].apply(lambda x: len(x)>=2)) & (df_pincho['Type'] == 'adverb')].sort_values(by='Similarity',ascending=False).head(size_of_candidates)
    candidate_adjectives = df_pincho.loc[df_pincho['Type']=='adjective'].sort_values(by='Similarity',ascending=False).head(size_of_candidates)

    #Adding kigo to the candidate list
    candidate_nouns['Kanji'][size_of_candidates-1] = kigo_choice
    candidate_nouns['Kana'][size_of_candidates-1] = kigo_kana
    candidate_nouns['Moras'][size_of_candidates-1] = kigo_size
    candidate_nouns['Similarity'][size_of_candidates-1] = compare(kigo_choice,kigo_choice)
    poetic_words = pd.DataFrame(['世界','目','愛','夢','涙','夜','空','心','風','月'],columns=['Kanji'])
    poetic_words['Kana'] = ['せかい','め','あい','ゆめ','なみだ','よる','そら','こころ','かぜ','つき']
    poetic_words['Moras'] = [mora(item) for item in poetic_words['Kana']]
    poetic_words['Type'] = ['noun']*poetic_words.shape[0]
    poetic_words['Similarity'] = [compare(kigo_choice,item) for item in poetic_words['Kanji']]
    poetic_words = poetic_words.sort_values('Similarity',ascending=False).head(size_of_candidates)

    persons = pd.DataFrame(['僕','君','私','人間','自分','貴方','彼女','あの人','俺','内'],columns=['Kanji'])
    persons['Kana'] = ['ぼく','きみ','わたし','にんげん','じぶん','あなた','かのじょ','あのひと','おれ','うち']
    persons['Moras'] = [mora(item) for item in persons['Kana']]
    persons['Type'] = ['noun']*persons.shape[0]
    persons['Similarity'] = [compare(kigo_choice,item) for item in persons['Kanji']]
    persons = persons.sort_values('Similarity',ascending=False).head(size_of_candidates)
    def order_heuristic(word1,word2):

        word1vec = ft.get_word_vector(word1)
        word2vec = ft.get_word_vector(word2)
        word1_first = cosine_similarity(np.expand_dims(word1vec,0),np.expand_dims(word1vec-word2vec,0))[0][0]
        word2_first = cosine_similarity(np.expand_dims(word2vec,0),np.expand_dims(word2vec-word1vec,0))[0][0]


        if (word1_first<word2_first):
            first_word = word1
            second_word = word2
        else:
            first_word = word2
            second_word = word1

        return first_word,second_word

    def grader(verse, print_scores = False):
        
        #Cos similarity score
        combinations = list(itertools.combinations(verse,2))
        scores = [compare(item[0],item[1]) for item in combinations]
        score = sum(scores)/(len(scores)+0.00001)

        #Punishing repeated characters, to favor variety
        allchars = [char for char in ''.join(verse)]
        repeats = np.array(list(Counter(allchars).values()))
        repeats = repeats[repeats>=2].sum()
        repeats_score = repeats/len(allchars)

        #Punishing words that are too uncommon or hard to read
        terms = list(verse)
        frequency_score=0
        for term in terms:
            if (df_pincho_raw['Kanji']==term).any():
                position = df_pincho_raw[df_pincho_raw['Kanji'] == term].index[0]
                frequency_score += (position/df_pincho_raw.shape[0])/2
            else:
                frequency_score += 0.5

        #Rewarding Kireji and ga aru, etc to compensate the fact that they will probably get a low cos similarity score.
        terms = list(verse)
        kireji_score=0
        kireji_etc = ['や','かな','がある','けり']
        if any(item in verse for item in kireji_etc):
            kireji_score+=0.4*np.random.rand()

        #Bias for single word verses
        bias = 0
        if len(list(verse))==1:
            bias+=0.8

        if print_scores:
            print(repeats_score,frequency_score,kireji_score)

        final_score = score - repeats_score - frequency_score + kireji_score + bias

        return final_score
    import itertools

    #First Verse

    candidate_sentences = []
    #Build candidate sentences:

    permutator = dict(zip(list(range(0,size_of_candidates**2)),list(itertools.product(list(range(0,size_of_candidates)), list(range(0,size_of_candidates))))))

    #Adverb-verb
    #list1 = list(candidate_adverbs['Kana'])
    #list2 = list(candidate_verbs['Kana'])

    #for i,item in enumerate(itertools.product(list1, list2)):
    #    if len(''.join(item))==5:
    #        id1,id2 = permutator[i][0],permutator[i][1]
    #        candidate_sentences.append(list(candidate_adverbs['Kanji'])[id1]+list(candidate_verbs['Kanji'])[id2])

    #Noun-verb

    list1 = list(candidate_nouns['Kana'])
    list2 = list(candidate_verbs['Kana'])

    for i,item in enumerate(itertools.product(list1, list2)):
        if mora(''.join(item))==5:
            id1,id2 = permutator[i][0],permutator[i][1]
            candidate_sentences.append((list(candidate_nouns['Kanji'])[id1],list(candidate_verbs['Kanji'])[id2]))

    #Adjective-noun

    list1 = list(candidate_adjectives['Kana'])
    list2 = list(candidate_nouns['Kana'])

    for i,item in enumerate(itertools.product(list1, list2)):

        if mora(''.join(item))==5:
            id1,id2 = permutator[i][0],permutator[i][1]
            candidate_sentences.append((list(candidate_adjectives['Kanji'])[id1],list(candidate_nouns['Kanji'])[id2]))

    #Noun-ya

    list1 = list(candidate_nouns['Kana'])
    list2 = len(list1)*['や']

    for i,item in enumerate(itertools.product(list1, list2)):
        if mora(''.join(item))==5:
            id1,id2 = permutator[i][0],permutator[i][1]
            candidate_sentences.append((list(candidate_nouns['Kanji'])[id1],'や'))

    #Noun-ni

    list1 = list(candidate_nouns['Kana'])
    list2 = len(list1)*['に']

    for i,item in enumerate(itertools.product(list1, list2)):
        if mora(''.join(item))==5:
            id1,id2 = permutator[i][0],permutator[i][1]
            candidate_sentences.append((list(candidate_nouns['Kanji'])[id1],'に'))

    #Person-no-poetic


    list1 = list(persons['Kana'])
    list2 = list(poetic_words['Kana'])

    for i,item in enumerate(itertools.product(list1, list2)):
        if mora(''.join(item))==4 and np.random.rand()>prob_poetic:
            id1,id2 = permutator[i][0],permutator[i][1]
            candidate_sentences.append((list(persons['Kanji'])[id1],'の',list(poetic_words['Kanji'])[id2]))

    #Noun-no-Noun

    list1 = list(candidate_nouns['Kana'])
    list2 = list(candidate_nouns['Kana'])

    for i,item in enumerate(itertools.product(list1, list2)):
        if mora(''.join(item))==4:
            id1,id2 = permutator[i][0],permutator[i][1]
            word1 = list(candidate_nouns['Kanji'])[id1]
            word2 = list(candidate_nouns['Kanji'])[id2]
            if word1 != word2:
            
                first, second = order_heuristic(word1,word2)
                candidate_sentences.append((first,'の',second))


    first_verse_candidate_sentences = list(set(candidate_sentences))
    #Second Verse

    candidate_sentences = []
    #Build candidate sentences:

    #This just keeps track of which permutation pairs correspond to which indexes after permuting.
    permutator = dict(zip(list(range(0,size_of_candidates**2)),list(itertools.product(list(range(0,size_of_candidates)), list(range(0,size_of_candidates))))))

    #list1 = list(candidate_adverbs['Kana'])
    #list2 = list(candidate_verbs['Kana'])

    #for i,item in enumerate(itertools.product(list1, list2)):
    #    if len(''.join(item))==5:
    #        id1,id2 = permutator[i][0],permutator[i][1]
    #        candidate_sentences.append(list(candidate_adverbs['Kanji'])[id1]+list(candidate_verbs['Kanji'])[id2])

    #Noun-verb

    list1 = list(candidate_nouns['Kana'])
    list2 = list(candidate_verbs['Kana'])

    for i,item in enumerate(itertools.product(list1, list2)):
        if mora(''.join(item))==7:
            id1,id2 = permutator[i][0],permutator[i][1]
            candidate_sentences.append((list(candidate_nouns['Kanji'])[id1],list(candidate_verbs['Kanji'])[id2]))

    #Noun-ni

    list1 = list(candidate_nouns['Kana'])
    list2 = len(list1)*['に']

    for i,item in enumerate(itertools.product(list1, list2)):
        if mora(''.join(item))==7:
            id1,id2 = permutator[i][0],permutator[i][1]
            candidate_sentences.append((list(candidate_nouns['Kanji'])[id1],'に'))

    #Person-no-poetic


    list1 = list(persons['Kana'])
    list2 = list(poetic_words['Kana'])

    for i,item in enumerate(itertools.product(list1, list2)):
        if mora(''.join(item))==6 and np.random.rand()>prob_poetic:
            id1,id2 = permutator[i][0],permutator[i][1]
            candidate_sentences.append((list(persons['Kanji'])[id1],'の',list(poetic_words['Kanji'])[id2]))

    #Noun-ya

    list1 = list(candidate_nouns['Kana'])
    list2 = len(list1)*['や']

    for i,item in enumerate(itertools.product(list1, list2)):
        if mora(''.join(item))==7:
            id1,id2 = permutator[i][0],permutator[i][1]
            candidate_sentences.append((list(candidate_nouns['Kanji'])[id1],'や'))

    #Noun-no-verb-keri
    list1 = list(candidate_nouns['Kana'])
    list2 = list(candidate_verbs['Kana'])

    for i,item in enumerate(itertools.product(list1, list2)):
        if mora(''.join(item))==4:
            id1,id2= permutator[i][0],permutator[i][1]

            verb_to_write = list(candidate_verbs['Kanji'])[id2]

            try:
                if verb_to_write[-2:] == 'する':
                    verb_to_write = verb_to_write[:-2]+'しり'

                else:
                    verb_u  = ['る','む','く','う','す','ぐ','つ']
                    verb_i  = ['り','み','き','い','し','ぎ','ち']
                    verb_ui = dict(zip(verb_u,verb_i))
                    verb_to_write = verb_to_write[:-1]+verb_ui[verb_to_write[-1]]
            
            except:
                pass

            candidate_sentences.append((list(candidate_nouns['Kanji'])[id1],'の',verb_to_write,'けり'))

    #Verb-keri
    list1 = list(candidate_nouns['Kana'])
    list2 = list(candidate_verbs['Kana'])

    for i,item in enumerate(itertools.product(list1, list2)):
        if mora(''.join(item))==5:
            id1,id2= permutator[i][0],permutator[i][1]

            verb_to_write = list(candidate_verbs['Kanji'])[id2]

            try:
                if verb_to_write[-2:] == 'する':
                    verb_to_write = verb_to_write[:-2]+'しり'

                else:
                    verb_u  = ['る','む','く','う','す','ぐ','つ']
                    verb_i  = ['り','み','き','い','し','ぎ','ち']
                    verb_ui = dict(zip(verb_u,verb_i))
                    verb_to_write = verb_to_write[:-1]+verb_ui[verb_to_write[-1]]
            
            except:
                pass

            candidate_sentences.append((list(candidate_nouns['Kanji'])[id1],verb_to_write,'けり'))

    #Adverb-ni-verb

    list1 = list(candidate_adverbs['Kana'])
    list2 = list(candidate_verbs['Kana'])

    for i,item in enumerate(itertools.product(list1, list2)):
        if mora(''.join(item))==6:
            id1,id2 = permutator[i][0],permutator[i][1]
            candidate_sentences.append((list(candidate_adverbs['Kanji'])[id1],'に',list(candidate_verbs['Kanji'])[id2]))

    #Adjective-noun

    list1 = list(candidate_adjectives['Kana'])
    list2 = list(candidate_nouns['Kana'])

    for i,item in enumerate(itertools.product(list1, list2)):
        if mora(''.join(item))==7:
            id1,id2 = permutator[i][0],permutator[i][1]
            candidate_sentences.append((list(candidate_adjectives['Kanji'])[id1],list(candidate_nouns['Kanji'])[id2]))

    #Noun-no-Noun

    list1 = list(candidate_nouns['Kana'])
    list2 = list(candidate_nouns['Kana'])

    for i,item in enumerate(itertools.product(list1, list2)):
        if mora(''.join(item))==6:
            id1,id2 = permutator[i][0],permutator[i][1]
            word1 = list(candidate_nouns['Kanji'])[id1]
            word2 = list(candidate_nouns['Kanji'])[id2]
            if word1 != word2:
            
                first, second = order_heuristic(word1,word2)
                candidate_sentences.append((first,'の',second))

    second_verse_candidate_sentences = list(set(candidate_sentences))
    #Third Verse

    candidate_sentences = []
    #Build candidate sentences:



    #This just keeps track of which permutation pairs correspond to which indexes after permuting.
    permutator = dict(zip(list(range(0,size_of_candidates**2)),list(itertools.product(list(range(0,size_of_candidates)), list(range(0,size_of_candidates))))))

    #Adverb-verb

    #list1 = list(candidate_adverbs['Kana'])
    #list2 = list(candidate_verbs['Kana'])

    #for i,item in enumerate(itertools.product(list1, list2)):
    #    if len(''.join(item))==5:
    #        id1,id2 = permutator[i][0],permutator[i][1]
    #        candidate_sentences.append(list(candidate_adverbs['Kanji'])[id1]+list(candidate_verbs['Kanji'])[id2])

    #Noun-verb

    list1 = list(candidate_nouns['Kana'])
    list2 = list(candidate_verbs['Kana'])

    for i,item in enumerate(itertools.product(list1, list2)):
        if mora(''.join(item))==5:
            id1,id2 = permutator[i][0],permutator[i][1]
            candidate_sentences.append((list(candidate_nouns['Kanji'])[id1],list(candidate_verbs['Kanji'])[id2]))

    #Adverb-ni-verb

    list1 = list(candidate_adverbs['Kana'])
    list2 = list(candidate_verbs['Kana'])

    for i,item in enumerate(itertools.product(list1, list2)):
        if mora(''.join(item))==4:
            id1,id2 = permutator[i][0],permutator[i][1]
            candidate_sentences.append((list(candidate_adverbs['Kanji'])[id1],'に',list(candidate_verbs['Kanji'])[id2]))

    #Person-no-poetic


    list1 = list(persons['Kana'])
    list2 = list(poetic_words['Kana'])

    for i,item in enumerate(itertools.product(list1, list2)):
        if mora(''.join(item))==4 and np.random.rand()>prob_poetic:
            id1,id2 = permutator[i][0],permutator[i][1]
            candidate_sentences.append((list(persons['Kanji'])[id1],'の',list(poetic_words['Kanji'])[id2]))

    #Adjective-noun

    list1 = list(candidate_adjectives['Kana'])
    list2 = list(candidate_nouns['Kana'])

    for i,item in enumerate(itertools.product(list1, list2)):
        if mora(''.join(item))==5:
            id1,id2 = permutator[i][0],permutator[i][1]
            candidate_sentences.append((list(candidate_adjectives['Kanji'])[id1],list(candidate_nouns['Kanji'])[id2]))

    #Noun-kana

    list1 = list(candidate_nouns['Kana'])
    list2 = len(list1)*['かな']

    for i,item in enumerate(itertools.product(list1, list2)):
        if mora(''.join(item))==5:
            id1,id2 = permutator[i][0],permutator[i][1]
            candidate_sentences.append((list(candidate_nouns['Kanji'])[id1],'かな'))

    #Noun-no-Noun

    list1 = list(candidate_nouns['Kana'])
    list2 = list(candidate_nouns['Kana'])

    for i,item in enumerate(itertools.product(list1, list2)):
        if mora(''.join(item))==4:
            id1,id2 = permutator[i][0],permutator[i][1]
            word1 = list(candidate_nouns['Kanji'])[id1]
            word2 = list(candidate_nouns['Kanji'])[id2]
            if word1 != word2:
            
                first, second = order_heuristic(word1,word2)
                candidate_sentences.append((first,'の',second))

    #Single word verses
    list1 = list(pd.concat([candidate_nouns['Kana'],candidate_verbs['Kana']]))
    list1_kanji = list(pd.concat([candidate_nouns['Kanji'],candidate_verbs['Kanji']]))

    for i, item in enumerate(list1):

        if mora(item)==5:
            candidate_sentences.append((list1_kanji[i],))

    #Noun-ga-aru

    list1 = list(candidate_nouns['Kana'])
    list2 = len(list1)*['がある']

    for i,item in enumerate(itertools.product(list1, list2)):
        if mora(''.join(item))==5:
            id1,id2 = permutator[i][0],permutator[i][1]
            candidate_sentences.append((list(candidate_nouns['Kanji'])[id1],'がある'))

    third_verse_candidate_sentences = list(set(candidate_sentences))
    size_of_grid = 7
    verse1 = sorted([[verse,grader(verse)] for verse in first_verse_candidate_sentences], key = lambda x: x[1],reverse=True)[:size_of_grid]
    verse2 = sorted([[verse,grader(verse)] for verse in second_verse_candidate_sentences], key = lambda x: x[1],reverse=True)[:size_of_grid]
    verse3 = sorted([[verse,grader(verse)] for verse in third_verse_candidate_sentences], key = lambda x: x[1],reverse=True)[:size_of_grid]
    def haiku_grader(haiku):
        haiku_string = ''.join((''.join(verse) for verse in haiku))

        #Cos similarity between verses
        #Sentence Embedding (Using Average Embedding of words, maybe in the future will use Vector Extrema,
        #but some initial tests didn't confirm the assumption that less common/ more contextual words have larger norms)
        #Need to do more tests
        sentence_vectors = [(np.sum(list(map(ft.get_word_vector,verse)),axis=0)/len(verse)) for verse in haiku]
        combinations = list(itertools.combinations(sentence_vectors,2))
        scores = [compare_sentence(item[0],item[1]) for item in combinations]
        randomizer = np.random.randint(low=-1,high=2)
        similarity_score = randomizer*(sum(scores)/(len(scores)+0.00001))

        #Punishing repeated characters, to favor variety
        allchars = [char for char in haiku_string]
        repeats = np.array(list(Counter(allchars).values()))
        repeats = repeats[repeats>=2].sum()
        repeats_score = repeats/len(allchars)+0.5

        #Rewarding one Kireji, but punishing simultaneous Kireji
        kireji_score=0
        kireji_etc = ['や','かな','けり']

        if any(item in haiku_string for item in kireji_etc):
            randomizer = np.random.randint(low=-1,high=2)
            kireji_score+=randomizer*0.45*np.random.rand()
      
        if sum(item in haiku_string for item in kireji_etc)>=2:
            kireji_score+=0.4
        final_score = 1-repeats_score-kireji_score+(similarity_score/10)

        #Reward using the kigo
        
        kigo_score=0
        if (kigo_choice in haiku_string):
            kigo_score+=0.5
        final_score = 1-repeats_score-kireji_score+kigo_score+(similarity_score/10)

        return final_score

    #Generating a Kanji 2 Kana dictionary to make the poems more easily readable
    kanji2kana = pd.concat([candidate_adjectives[['Kanji','Kana']],
    candidate_adverbs[['Kanji','Kana']],
    candidate_nouns[['Kanji','Kana']],
    candidate_verbs[['Kanji','Kana']],
    persons[['Kanji','Kana']],
    poetic_words[['Kanji','Kana']]])
    additions = ['や','かな','けり','を','に','の']
    for item in additions:
        kanji2kana = kanji2kana.append({'Kanji': item,'Kana': item,},ignore_index=True)

    kanji2kana = dict(zip(kanji2kana['Kanji'],kanji2kana['Kana']))

    final_haiku_selection = []
    final_haiku_readings = []
    
    for i,item in enumerate(itertools.product([item[0] for item in verse1], [item[0] for item in verse2], [item[0] for item in verse3])):
        haiku = (''.join(verse) for verse in item)
        haiku_kana = []
        n_words = 0
        for verse in item:
            readings = [kanji2kana[word] for word in verse]
            haiku_kana.append(''.join(readings))
            n_words+=len(verse)
        
        grades = haiku_grader(item)
        final_haiku_selection.append([','.join(haiku),grades,n_words])
        final_haiku_readings.append([','.join(haiku_kana),grades,n_words])

    final_haiku_selection = sorted(final_haiku_selection, key = lambda x: x[1],reverse=True)
    final_haiku_readings = sorted(final_haiku_readings, key = lambda x: x[1],reverse=True)
    
    global total_number_of_words

    total_number_of_words+= sum([item[2] for item in final_haiku_readings[:4]])
    print(total_number_of_words)

    
    final_kanjis = final_haiku_selection[:4]
    final_kanas = final_haiku_readings[:4]
    if len(final_kanjis)==4:
        for item in zip(final_kanjis,final_kanas):
            textfile.write(kigo_choice+';'+item[0][0]+';'+item[1][0]+'\n')
    else:
        raise ValueError('The code got tired.')
    


In [ ]:
#Generating (a man can hope) all the haiku

#Filtering kigo that are too big for the code to handle
all_kigo = kigo[['japanese','hiragana']][:]
all_kigo['Mora'] = [mora(item) for item in all_kigo['hiragana']]
filtered_kigo = all_kigo.loc[(all_kigo['Mora'] <=7 )].loc[(all_kigo['Mora'] >0 )]


In [ ]:
hold_all = []
total_number_of_words = 0

#kigo_choice = '鴨'
#kigo_kana = 'みぞれ'
#kigo_size = mora(kigo_kana) 

#test = create_haiku(kigo_choice,kigo_kana,kigo_size)


In [ ]:
for index, row in tqdm(filtered_kigo.iterrows()):
    kigo_choice = row['japanese']
    kigo_kana = row['hiragana']
    kigo_size = row['Mora']
    textfile = open("thehaiku.txt", "a")
    try:
        create_haiku(kigo_choice,kigo_kana,kigo_size)
    except:
        pass
    textfile.close()
    

1it [00:33, 33.55s/it]

25


4it [01:35, 23.17s/it]

49


5it [02:02, 24.49s/it]

...

18762


1182it [7:23:39, 23.45s/it]

18790


1183it [7:24:07, 22.53s/it]

18816
